In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import numpy as np
import requests
import pandas as pd
from io import StringIO
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import Dataset
import logging

from datasets import load_dataset

#load train data
import pandas as pd

import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import Dataset
import logging

from datasets import load_dataset

raw_datasets  = load_dataset('nyu-mll/glue', 'mrpc')




/home/ubuntu/miniconda3/envs/emnlp_2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
#from roberta import RobertaForSequenceClassification


model_name = "microsoft/deberta-v3-base"

#config.num_labels=2
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = 'left'

/home/ubuntu/miniconda3/envs/emnlp_2/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
from datasets import DatasetDict

mask_token = tokenizer.mask_token

def generate_prompt(data_point):
    """
    Generates a prompt for evaluating the humor intensity of an edited headline.
    Args:
        data_point (dict): A dictionary containing 'original', 'edit', and 'meanGrade'.
    Returns:
        str: The formatted prompt as a string.
    """
    return f"""# Sentence-1:: {data_point['sentence1']}. # Sentence-2: {data_point['sentence2']} # Output: The relationship is"""  # noqa: E501


# Assuming `dataset` is your DatasetDict
def add_label_column(example):

    example['labels'] = example['label']
  
    example['input'] = generate_prompt(example)

    
    return example

# Map the function over train and validation datasets

train_data = raw_datasets['train'].map(add_label_column)
val_data = raw_datasets['validation'].map(add_label_column)

# Remove unnecessary columns

# Inspect the updated datasets
print("Train Dataset:", train_data)
print("Validation Dataset:", val_data)

Train Dataset: Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'labels', 'input'],
    num_rows: 3668
})
Validation Dataset: Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'labels', 'input'],
    num_rows: 408
})


In [6]:
train_data['label'][10]

0

In [7]:
from transformers import AutoTokenizer, DataCollatorWithPadding


tokenizer.padding_side = 'left'


# col_to_delete = ['idx']
col_to_delete =  ['sentence1', 'sentence2', 'label', 'idx']

mask_token = tokenizer.mask_token
def preprocessing_function(examples):
   
    return tokenizer(examples['input'], truncation=True, max_length=512)

tokenized_train_data = train_data.map(preprocessing_function, batched=True, remove_columns=col_to_delete)
tokenized_val_data = val_data.map(preprocessing_function, batched=True, remove_columns=col_to_delete)
# llama_tokenized_datasets = llama_tokenized_datasets.rename_column("target", "label")
tokenized_train_data.set_format("torch")
tokenized_val_data.set_format("torch")

# Data collator for padding a batch of examples to the maximum length seen in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 408/408 [00:00<00:00, 6348.69 examples/s]


In [8]:
tokenizer.decode(tokenized_train_data['input_ids'][10])


'[CLS] # Sentence-1:: Legislation making it harder for consumers to erase their debts in bankruptcy court won overwhelming House approval in March .. # Sentence-2: Legislation making it harder for consumers to erase their debts in bankruptcy court won speedy , House approval in March and was endorsed by the White House . # Output: The relationship is[SEP]'

In [9]:
val_data

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'labels', 'input'],
    num_rows: 408
})

In [10]:
all_lengths = [len(ids) for ids in tokenized_train_data['input_ids']]
mx = max(all_lengths)
mx


119

In [11]:
count = sum(len(ids) > 512 for ids in tokenized_train_data['input_ids'])
print(count)


0


In [12]:
import torch
import torch.nn as nn
from transformers import RobertaForSequenceClassification
from transformers.activations import ACT2FN
import random

from transformers import AutoModelForSequenceClassification
config = AutoConfig.from_pretrained(model_name)
config.mask_token_id = tokenizer.mask_token_id
config.num_labels = 2

model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
import RoCoFT

RoCoFT.PEFT(model, method='column', rank=3) 
#targets=['key', 'value', 'dense', 'query'])

In [14]:

import evaluate
import numpy as np
from sklearn import metrics
import torch
import numpy as np

def compute_metrics(eval_pred):


    logits, labels = eval_pred # eval_pred is the tuple of predictions and labels returned by the model
    predictions = np.argmax(logits, axis=-1)
    
    precision = metrics.precision_score(labels, predictions, average="macro")
    recall = metrics.recall_score(labels, predictions, average="macro")
    f1 = metrics.f1_score(labels, predictions, average="macro")
    accuracy = metrics.accuracy_score(labels, predictions)
    
    return {"precision": precision, "recall": recall, "f1-score": f1, 'accuracy': accuracy}

In [16]:
from transformers import TrainingArguments, Trainer

import time
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='dir',
    learning_rate=3e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.0,
    eval_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    save_steps=10000000,
    logging_steps=100,
   
    load_best_model_at_end=True,
    lr_scheduler_type="cosine",  # You can choose from 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', etc.
    warmup_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,

    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [17]:
trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
100,0.633000,0.607873,0.716539,0.535467,0.485615,0.700980
200,0.543500,0.385278,0.815701,0.766483,0.783341,0.825980
300,0.492600,0.403419,0.813518,0.810869,0.812165,0.838235
400,0.415900,0.396744,0.862273,0.790364,0.813301,0.852941
500,0.373800,0.286004,0.877103,0.851171,0.862280,0.884804
600,0.305900,0.396394,0.870220,0.805868,0.827703,0.862745
700,0.279200,0.393171,0.883941,0.817204,0.840010,0.872549
800,0.210500,0.388087,0.876417,0.801700,0.825747,0.862745
900,0.225400,0.350248,0.901217,0.836292,0.859247,0.887255
1000,0.176400,0.332657,0.883190,0.856839,0.868140,0.889706


TrainOutput(global_step=1150, training_loss=0.3357609811036483, metrics={'train_runtime': 380.1067, 'train_samples_per_second': 48.25, 'train_steps_per_second': 3.025, 'total_flos': 3673732172352.0, 'train_loss': 0.3357609811036483, 'epoch': 5.0})

In [16]:
train_data

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'labels', 'input'],
    num_rows: 3668
})